### Creating data-asset using relative path to datastore

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

In [2]:
keyVaultName = "mobilepricinga6945442583"
KVUri = f"https://{keyVaultName}.vault.azure.net"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

In [4]:
credential = DefaultAzureCredential()

In [5]:
subs_id = client.get_secret("subscription-id").value
rg_name = client.get_secret("ml-resource-group").value
ws_name = client.get_secret("ml-workspace-name").value

In [6]:
ml_client = MLClient(
    credential=credential,
    subscription_id=subs_id,
    resource_group_name=rg_name,
    workspace_name=ws_name,
)

In [11]:
## create a data-store
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration
name = "mobile_pricing_v2"
description = "training dataset stored in blob store"
account_name = "shareddatalakedev"
container_name = "silver"
account_key = client.get_secret("storage-account-key")
store = AzureBlobDatastore(
    name=name,
    description=description,
    account_name=account_name,
    container_name=container_name,
    protocol="https",
    credentials=AccountKeyConfiguration(account_key=account_key),
properties = {"subscription_id":subs_id,"resource_group_name":rg_name}
)

ml_client.create_or_update(store)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'mobile_pricing_v2', 'description': 'training dataset stored in blob store', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/0aa1c63a-7a46-403c-91e4-8ec91264bc42/resourceGroups/rg-mobile-pricing-dev/providers/Microsoft.MachineLearningServices/workspaces/mobile-pricing-azml-dev/datastores/mobile_pricing_v2', 'Resource__source_path': None, 'base_path': '/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/consulting/setu/mlops_bitbucket/content/01_Mlops_Using_Cloud_Tools/datasets_azure_ml/notebooks', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x120ea8b80>, 'credentials': {'type': 'account_key'}, 'container_name': 'silver', 'account_name': 'shareddatalakedev', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [13]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

uri = "azureml://datastores/mobile_pricing_v2/paths/mobile-pricing-train/01_01_24/train.csv"
VERSION="1"
mobile_pricing_train_data = Data(path=uri,
                         type=AssetTypes.URI_FILE,
    description="Train data for mobile pricing",
    name="mobile_pricing_train_data_v1",
    version=VERSION,
)

In [14]:
ml_client.data.create_or_update(mobile_pricing_train_data)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'mobile_pricing_train_data_v1', 'description': 'Train data for mobile pricing', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/0aa1c63a-7a46-403c-91e4-8ec91264bc42/resourceGroups/rg-mobile-pricing-dev/providers/Microsoft.MachineLearningServices/workspaces/mobile-pricing-azml-dev/data/mobile_pricing_train_data_v1/versions/1', 'Resource__source_path': None, 'base_path': '/Users/gunnvantsaini/Library/CloudStorage/OneDrive-Personal/consulting/setu/mlops_bitbucket/content/01_Mlops_Using_Cloud_Tools/datasets_azure_ml/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x120c4b9a0>, 'serialize': <msrest.serialization.Serializer object at 0x1210c48b0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/0aa1c63a-7a4